<a href="https://colab.research.google.com/github/Noahkandie/Indigenous-Research-Storytelling/blob/main/Indigenous_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import re
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords

In [2]:
path = 'indigenous_data.csv'
df = pd.read_csv(path)
df.head()

,text,quote/text,author/description,author/followers,author/favouritesCount,author/createdAt,source,quote/viewCount,quoteCount,twitterUrl,type,url,createdAt,author/location,retweetCount,replyCount,likeCount
0,Our latest newspaper edition @Eastleighvoice h...,NaN,"Print journalist? I write | Yes,in my mundane,...",281477,43566,Tue Mar 27 14:15:56 +0000 2018,Twitter for iPhone,NaN,2,https://twitter.com/Honeyfarsafi/status/179660...,tweet,https://x.com/Honeyfarsafi/status/179660743141...,Fri May 31 18:19:25 +0000 2024,"Nairobi, Kenya",5.0,NaN,56.0
1,@DiamondDk20 Mine is replacement. Basically ju...,NaN,"Print journalist? I write | Yes,in my mundane,...",281477,43566,Tue Mar 27 14:15:56 +0000 2018,Twitter for iPhone,NaN,0,https://twitter.com/Honeyfarsafi/status/179654...,tweet,https://x.com/Honeyfarsafi/status/179654792626...,Fri May 31 14:22:58 +0000 2024,"Nairobi, Kenya",0.0,NaN,16.0
2,The government has totally shocked me today. I...,NaN,"Print journalist? I write | Yes,in my mundane,...",281477,43566,Tue Mar 27 14:15:56 +0000 2018,Twitter for iPhone,NaN,17,https://twitter.com/Honeyfarsafi/status/179654...,tweet,https://x.com/Honeyfarsafi/status/179654650881...,Fri May 31 14:17:20 +0000 2024,"Nairobi, Kenya",122.0,NaN,1184.0
3,@maanxur Imagine 🤣🤣🤣🤣,NaN,"Print journalist? I write | Yes,in my mundane,...",281477,43566,Tue Mar 27 14:15:56 +0000 2018,Twitter for iPhone,NaN,0,https://twitter.com/Honeyfarsafi/status/179651...,tweet,https://x.com/Honeyfarsafi/status/179651695954...,Fri May 31 12:19:55 +0000 2024,"Nairobi, Kenya",0.0,NaN,1.0
4,"Please very guided, I get hundreds of calls da...",NaN,"Print journalist? I write | Yes,in my mundane,...",281477,43566,Tue Mar 27 14:15:56 +0000 2018,Twitter for iPhone,NaN,1,https://twitter.com/Honeyfarsafi/status/179649...,tweet,https://x.com/Honeyfarsafi/status/179649741360...,Fri May 31 11:02:15 +0000 2024,"Nairobi, Kenya",26.0,NaN,99.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8439 entries, 0 to 8438
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   text                    8439 non-null   object 
 1   quote/text              1765 non-null   object 
 2   author/description      8439 non-null   object 
 3   author/followers        8439 non-null   int64  
 4   author/favouritesCount  8439 non-null   int64  
 5   author/createdAt        8439 non-null   object 
 6   source                  8439 non-null   object 
 7   quote/viewCount         1811 non-null   float64
 8   quoteCount              8439 non-null   int64  
 9   twitterUrl              8166 non-null   object 
 10  type                    8439 non-null   object 
 11  url                     8439 non-null   object 
 12  createdAt               8439 non-null   object 
 13  author/location         8439 non-null   object 
 14  retweetCount            7813 non-null   

In [4]:
df.columns

Index(['text', 'quote/text', 'author/description', 'author/followers',
       'author/favouritesCount', 'author/createdAt', 'source',
       'quote/viewCount', 'quoteCount', 'twitterUrl', 'type', 'url',
       'createdAt', 'author/location', 'retweetCount', 'replyCount',
       'likeCount'],
      dtype='object')

In [5]:
df.isnull().sum()

,0
text,0
quote/text,6674
author/description,0
author/followers,0
author/favouritesCount,0
author/createdAt,0
source,0
quote/viewCount,6628
quoteCount,0
twitterUrl,273


In [6]:
missing_percentages = df.isnull().mean() * 100
sorted_missing_percentages = missing_percentages.sort_values(ascending=False)

print(sorted_missing_percentages)

quote/text                79.085200
quote/viewCount           78.540111
replyCount                13.295414
likeCount                 11.600901
retweetCount               7.417941
twitterUrl                 3.234980
type                       0.000000
author/location            0.000000
createdAt                  0.000000
url                        0.000000
text                       0.000000
source                     0.000000
author/createdAt           0.000000
author/favouritesCount     0.000000
author/followers           0.000000
author/description         0.000000
quoteCount                 0.000000
dtype: float64


In [7]:
df.describe()

,author/followers,author/favouritesCount,quote/viewCount,quoteCount,retweetCount,replyCount,likeCount
count,8.439000e+03,8439.000000,1.811000e+03,8439.000000,7813.000000,7317.000000,7460.000000
mean,1.525786e+06,29674.413082,6.190885e+05,19.195047,486.120696,133.217576,2017.008177
std,2.582100e+06,19993.504623,4.581485e+06,65.585972,1120.875864,362.173093,3700.418257
min,2.814680e+05,144.000000,1.507000e+03,0.000000,0.000000,0.000000,0.000000
25%,2.814770e+05,1983.000000,2.913950e+04,0.000000,2.000000,3.000000,23.000000
50%,2.814880e+05,43566.000000,1.033040e+05,2.000000,66.000000,30.000000,347.000000
75%,2.916100e+05,43566.000000,2.972975e+05,13.000000,331.000000,127.000000,2302.500000
max,6.891058e+06,43566.000000,1.657991e+08,2027.000000,17861.000000,9414.000000,40058.000000


## Data cleaning

In [8]:
# Load stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    """
    Cleans the input text by:
    - Removing URLs
    - Removing special characters and numbers
    - Converting to lowercase
    - Removing punctuation
    - Removing stopwords
    """
    # Remove URLs
    text = str(text)
    text = re.sub(r'http\S+|www.\S+th', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize and remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# Define the mapping of Twitter URLs to names
url_mapping = {
    "https://twitter.com/Honeyfarsafi": "Hanifa",
    "https://twitter.com/WilliamsRuto": "WSR",
    "https://twitter.com/MoraraKebasoSnr": "Morara"
}

# Replace the URLs with the corresponding names
df['users'] = df['author/twitterUrl'].replace(url_mapping)

KeyError: 'author/twitterUrl'

In [ ]:
# Apply the cleaning function
df['cleaned_text'] = df['text'].apply(clean_text)


## User analysis

In [ ]:
# Plot the top 10 active users
user_activity = df['users'].value_counts()

plt.figure(figsize=(10, 6))
user_activity.head(10).plot(kind='bar', color='purple', edgecolor='black')
plt.title('Users', fontsize=16)
plt.xlabel('User', fontsize=12)
plt.ylabel('Number of Tweets', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()


## Time

In [ ]:
# Convert 'createdAt' to datetime and separate into date and time
df['createdAt'] = pd.to_datetime(df['createdAt'], format='%a %b %d %H:%M:%S %z %Y')

df['date'] = df['createdAt'].dt.date
df['time'] = df['createdAt'].dt.time

# Extract year, month, and day
df['year'] = df['createdAt'].dt.year
df['month'] = df['createdAt'].dt.month
df['day'] = df['createdAt'].dt.day

In [ ]:
# Format date as DD/MM/YYYY and time as HH:MM
# df['date'] = df['date'].dt.strftime('%d/%m/%Y')
# df['time'] = df['time'].dt.strftime('%H:%M')
df[['date', 'time']]

In [ ]:
df['month'].nunique()

In [ ]:
# Group by date and count the number of tweets per day
tweets_by_date = df.groupby(df['createdAt'].dt.date).size()

# Plot the data
plt.figure(figsize=(10, 6))
tweets_by_date.plot(kind='line', marker='o', color='blue', linewidth=2)
plt.title('Number of Tweets Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Tweets', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
# Group by user and date to count tweets per day

user_trends = df.groupby(['users', df['createdAt'].dt.date]).size().reset_index(name='tweet_count')

# Pivot the data to create a time series for each user
user_trends_pivot = user_trends.pivot(index='createdAt', columns='users', values='tweet_count').fillna(0)

# Plot trends for the top 3 most active users
top_users = user_activity.head(3).index
user_trends_pivot[top_users].plot(figsize=(12, 6))
plt.title('Tweet Trends Over Time (Top 3 Users)', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Tweets', fontsize=12)
plt.legend(title='User', fontsize=10)
plt.grid(axis='both', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## n-Gram

In [ ]:
df.createdAt

In [ ]:
# Function to generate n-grams
def generate_ngrams(texts, n):
    vectorizer = CountVectorizer(ngram_range=(n, n), stop_words='english')
    ngrams = vectorizer.fit_transform(texts)
    ngram_counts = ngrams.toarray().sum(axis=0)
    ngram_features = vectorizer.get_feature_names_out()
    ngram_counter = Counter(dict(zip(ngram_features, ngram_counts)))
    return ngram_counter.most_common(20)  # Top 20 n-grams




In [ ]:
# Generate bigrams
bigrams = generate_ngrams(df['cleaned_text'], 2)
print("\nTop 20 Bigrams:")
for bigram, count in bigrams:
    print(f"{bigram}: {count}")



In [ ]:
# Generate trigrams
trigrams = generate_ngrams(df['cleaned_text'], 3)
print("\nTop 20 Trigrams:")
for trigram, count in trigrams:
    print(f"{trigram}: {count}")

In [ ]:
# Function to generate word clouds with custom colors
def generate_wordcloud(data, title, colormap):
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap=colormap).generate(" ".join(data))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16)
    plt.show()

## Sentiment analysis

In [ ]:
# Sentiment Analysis
df['polarity'] = df['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['subjectivity'] = df['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [ ]:
# Define sentiment categories
def categorize_sentiment(polarity):
    if polarity < 0:
        return 'negative'
    elif polarity == 0:
        return 'neutral'
    else:
        return 'positive'

df['sentiment_category'] = df['polarity'].apply(categorize_sentiment)

In [ ]:
# General Word Cloud (Viridis)
generate_wordcloud(df['cleaned_text'], "General Word Cloud", colormap='viridis')



In [ ]:
# Negative sentiment wordcloud
negative_text = df[df['sentiment_category'] == 'negative']['cleaned_text']
generate_wordcloud(negative_text, "Negative Sentiment Word Cloud", colormap='Reds')

In [ ]:
# Neutral sentiment wordcloud
neutral_text = df[df['sentiment_category'] == 'neutral']['cleaned_text']
generate_wordcloud(neutral_text, "Neutral Sentiment Word Cloud", colormap='Greys')

In [ ]:
# Positive wordcloud
positive_text = df[df['sentiment_category'] == 'positive']['cleaned_text']
generate_wordcloud(positive_text, "Positive Sentiment Word Cloud", colormap='Greens')

In [ ]:
# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 6), sharey=True)

# Polarity distribution
sns.histplot(df['polarity'], kde=True, bins=30, color='blue', ax=axes[0])
axes[0].set_title("Polarity Distribution")
axes[0].set_xlabel("Polarity")
axes[0].set_ylabel("Frequency")

# Subjectivity distribution
sns.histplot(df['subjectivity'], kde=True, bins=30, color='green', ax=axes[1])
axes[1].set_title("Subjectivity Distribution")
axes[1].set_xlabel("Subjectivity")

# Adjust layout
plt.tight_layout()
plt.suptitle("Sentiment Distribution", fontsize=16, y=1.02)
plt.show()

In [ ]:
# Generate word clouds for each author
authors = df['users'].unique()
for author in authors:
    text = ' '.join(df[df['users'] == author]['cleaned_text'])
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f"WordCloud for {author}")
    plt.axis('off')
    plt.show()


### Sentiment analysis grouped by user

In [ ]:
# Group by user and calculate mean sentiment polarity
user_sentiment = df.groupby('users')['polarity'].mean().sort_values(ascending=False)

# Plot the top 10 users with the most positive sentiment
plt.figure(figsize=(10, 6))
user_sentiment.head(10).plot(kind='bar', color='teal', edgecolor='black')
plt.title('Top 10 Users with Most Positive Sentiment', fontsize=16)
plt.xlabel('User', fontsize=12)
plt.ylabel('Average Polarity', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()

# Plot the top 10 users with the most negative sentiment
plt.figure(figsize=(10, 6))
user_sentiment.tail(10).plot(kind='bar', color='crimson', edgecolor='black')
plt.title('Top 10 Users with Most Negative Sentiment', fontsize=16)
plt.xlabel('User', fontsize=12)
plt.ylabel('Average Polarity', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()


## Mentions analysis

In [ ]:
# Extract mentions from text
def extract_mentions(text):
    """
    Extracts all @mentions from the given text.
    Handles potential float values in the 'text' column.
    """
    if isinstance(text, str):
        return [mention for mention in text.split() if mention.startswith('@')]
    else:
        return []

# Apply the extraction function to the 'text' column
df['mentions'] = df['text'].apply(extract_mentions)

# Flatten the list of mentions for analysis
all_mentions = [mention for mentions in df['mentions'] for mention in mentions]

# Count occurrences of each mentioned user
mention_counts = Counter(all_mentions)

# Create a DataFrame of the most mentioned users
mention_analysis = pd.DataFrame(mention_counts.items(), columns=['User', 'Count']).sort_values(by='Count', ascending=False)

# Display the top 10 most mentioned users
print(mention_analysis.head(10))

## User Interaction Analysis

In [ ]:
# Extract mentions from tweets
df['mentions'] = df['cleaned_text'].str.findall(r'@\w+')

# Explode mentions into separate rows
mentions_df = df.explode('mentions')

# Count mentions by user
mention_counts = mentions_df['mentions'].value_counts()

# Visualize top mentions
mention_counts.head(10).plot(kind='bar', figsize=(12, 6), color='coral')
plt.title("Top Mentioned Users")
plt.xlabel("User")
plt.ylabel("Count")
plt.show()


In [ ]:
import itertools
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt

# Extract mentions per row (already done previously)
df['mentions'] = df['text'].apply(extract_mentions)

# Generate occurrence pairs
co_mention_pairs = [
    pair for mentions in df['mentions'] if len(mentions) > 1 for pair in itertools.combinations(sorted(mentions), 2)]

# Count occurrences
co_mention_counts = Counter(co_mention_pairs)

# Create a df for comentions
co_mention_df = pd.DataFrame(
    [(pair[0], pair[1], count) for pair, count in co_mention_counts.items()],
    columns=['User1', 'User2', 'Count']
).sort_values(by='Count', ascending=False)

# Display top comentions
print(co_mention_df.head(10))

## Retweets vs original tweets

In [ ]:
# Identify retweets (tweets starting with 'RT @')
df['is_retweet'] = df['text'].str.startswith('RT @')

# Count retweets vs original tweets
retweet_counts = df['is_retweet'].value_counts()

# Plot the data
plt.figure(figsize=(8, 5))
retweet_counts.plot(kind='bar', color=['skyblue', 'orange'], edgecolor='black')
plt.title('Retweets vs Original Tweets', fontsize=16)
plt.xticks(ticks=[0, 1], labels=['Original Tweets', 'Retweets'], rotation=0)
plt.ylabel('Number of Tweets', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## Retweet influence by user

In [ ]:
df['retweeted_user'] = df['text'].str.extract(r'^RT @(\w+):')

# Count the most retweeted users and plot the top retweeted users
retweeted_user_counts = df['retweeted_user'].value_counts()

plt.figure(figsize=(10, 6))
retweeted_user_counts.head(10).plot(kind='bar', color='darkgreen', edgecolor='black')
plt.title('Top 10 most retweeted users', fontsize=16)
plt.xlabel('Retweeted user', fontsize=12)
plt.ylabel('Number of Retweets', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()

## Hashtag Analysis

In [ ]:
# Extract hashtags
df['hashtags'] = df['text'].str.findall(r'#\w+')

# Flatten the list of hashtags and count occurrences
all_hashtags = pd.Series([hashtag for hashtags in df['hashtags'] for hashtag in hashtags])
hashtag_counts = all_hashtags.value_counts()

# Plot the top 10 hashtags
plt.figure(figsize=(10, 6))
hashtag_counts.head(10).plot(kind='bar', color='orange', edgecolor='black')
plt.title('Top 10 Hashtags', fontsize=16)
plt.xlabel('Hashtag', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()


## LDA

In [ ]:
# Topic Modeling with LDA
vectorizer = CountVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['cleaned_text'])

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)


In [ ]:
# Display topics
def display_topics(model, feature_names, no_top_words):
    for idx, topic in enumerate(model.components_):
        print(f"Topic {idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

print("\nLDA Topics:")
display_topics(lda, vectorizer.get_feature_names_out(), 10)

In [ ]:
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import numpy as np

# Train a Word2Vec model
tokenized_text = [text.split() for text in df['cleaned_text']]
w2v_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=5, workers=4)

# Extract word vectors
word_vectors = np.array([w2v_model.wv[word] for word in w2v_model.wv.key_to_index])
word_labels = list(w2v_model.wv.key_to_index)

tsne = TSNE(n_components=2, random_state=42, perplexity=min(22, len(word_vectors))) # set perplexity to less than n_samples
reduced_vectors = tsne.fit_transform(word_vectors)

# Plot word embeddings
plt.figure(figsize=(14, 10))
plt.scatter(reduced_vectors[:, 0], reduced_vectors[:, 1], alpha=0.7)
for i, label in enumerate(word_labels):
    plt.annotate(label, (reduced_vectors[i, 0], reduced_vectors[i, 1]), fontsize=8, alpha=0.75)
plt.title("Word Embedding Visualization (t-SNE)")
plt.show()


In [ ]:
# Reduce dimensions using t-SNE, setting perplexity less than n_samples
tsne = TSNE(n_components=2, random_state=42, perplexity=min(22, len(word_vectors))) # set perplexity to less than n_samples
reduced_vectors = tsne.fit_transform(word_vectors)
reduced_vectors

## Topics cluster analysis

In [ ]:


# Convert text to TF-IDF matrix
vectorizer = TfidfVectorizer(max_features=500)
tfidf_matrix = vectorizer.fit_transform(df['clean_text'])

# K-Means clustering
kmeans = KMeans(n_clusters=5, random_state=42).fit(tfidf_matrix)
df['Cluster'] = kmeans.labels_

# Analyze themes in each cluster
for i in range(5):
    cluster_texts = ' '.join(df[df['Cluster'] == i]['clean_text'])
    print(f"Cluster {i} Top Words:")
    print(pd.Series(cluster_texts.split()).value_counts().head(10))


## Thematic Grouping by Keywords

In [ ]:
# Define thematic keywords
themes = {
    'Politics and Leadership': ['kenya', 'president', 'government', 'ruto', 'state', 'african', 'change'],
    'Economy and Development': ['project', 'money', 'change', 'ceremony', 'water', 'development'],
    'Security and Law': ['security', 'police', 'court', 'state', 'people'],
    'Culture and Community': ['cultural', 'family', 'church', 'people', 'kenyans', 'nairobi'],
    'Education and Social Services': ['school', 'county', 'people', 'taveta', 'taita', 'kwale']
}

# Create a new column for theme
def assign_theme(text):
    for theme, keywords in themes.items():
        if any(keyword in text.lower() for keyword in keywords):
            return theme
    return 'other'

df['theme'] = df['text'].apply(assign_theme)

# Count tweets by theme
theme_counts = df['theme'].value_counts()

# Plot the themes
plt.figure(figsize=(8, 5))
theme_counts.plot(kind='bar', color='green', edgecolor='black')
plt.title('Tweet Themes', fontsize=16)
plt.xlabel('Theme', fontsize=12)
plt.ylabel('Number of Tweets', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()


### Sentiment Analysis by Themes

In [ ]:
# Group by thematic keywords or hashtags
themes = ['economy', 'leadership', 'youth', 'Kenya', 'progress']  # Define themes
theme_sentiments = {}

for theme in themes:
    theme_tweets = df[df['cleaned_text'].str.contains(theme, case=False, na=False)]
    avg_polarity = theme_tweets['polarity'].mean()
    avg_subjectivity = theme_tweets['subjectivity'].mean()
    theme_sentiments[theme] = {'Polarity': avg_polarity, 'Subjectivity': avg_subjectivity}

theme_sentiments_df = pd.DataFrame(theme_sentiments).T
print(theme_sentiments_df)


### Temporal analysis of hashtags

In [ ]:
# Extract hashtags
df['hashtags'] = df['cleaned_text'].str.findall(r'#\w+')

# Explode hashtags into separate rows
hashtag_df = df[['created_at', 'hashtags']].explode('hashtags')
hashtag_df['date'] = pd.to_datetime(df['created_at']).dt.date

# Group by date and hashtag
hashtag_trends = hashtag_df.groupby(['date', 'hashtags']).size().reset_index(name='count')

# Visualize trends for top hashtags
top_hashtags = hashtag_trends['hashtags'].value_counts().nlargest(5).index
filtered_trends = hashtag_trends[hashtag_trends['hashtags'].isin(top_hashtags)]

plt.figure(figsize=(12, 6))
sns.lineplot(data=filtered_trends, x='date', y='count', hue='hashtags')
plt.title("Hashtag Trends Over Time")
plt.xlabel("Date")
plt.ylabel("Count")
plt.show()


## Co-occurrence Network Analysis

In [ ]:
from itertools import combinations
import networkx as nx
from collections import Counter

# Tokenize text and get co-occurring pairs
co_occurrence = []
for tokens in df['cleaned_text'].apply(lambda x: x.split()):
    co_occurrence.extend(combinations(tokens, 2))

# Count co-occurrence
co_occurrence_counts = Counter(co_occurrence)

# Create graph
G = nx.Graph()
for pair, weight in co_occurrence_counts.items():
    G.add_edge(pair[0], pair[1], weight=weight)

# Visualize using NetworkX (or save for Gephi)
plt.figure(figsize=(12, 12))
nx.draw_networkx(G, with_labels=True, node_size=50, font_size=8, edge_color='gray')
plt.title("Word co-occurrence network")
plt.show()
